In [ ]:
#export
from fastai2.basics import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
#default_exp callback.hook

# Model hooks

> Callback and helper function to add hooks in models

In [ ]:
from fastai2.test_utils import *

## What are hooks?

Hooks are functions you can attach to a particular layer in your model and that will be executed in the foward pass (for forward hooks) or backward pass (for backward hooks). Here we begin with an introduction around hooks, but you should jump to `HookCallback` if you quickly want to implement one (and read the following example `ActivationStats`).

Forward hooks are functions that take three arguments: the layer it's applied to, the input of that layer and the output of that layer.

In [ ]:
tst_model = nn.Linear(5,3)
def example_forward_hook(m,i,o): print(m,i,o)
    
x = torch.randn(4,5)
hook = tst_model.register_forward_hook(example_forward_hook)
y = tst_model(x)
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([[ 0.1946, -0.8289, -0.1138, -1.6296,  0.4023],
        [ 1.1464,  1.0207, -0.7386,  0.6931,  0.5356],
        [-0.6210, -0.1260,  0.2896,  0.6386,  0.5146],
        [-0.2803, -1.4319,  1.9844, -0.3058,  1.3207]]),) tensor([[-0.4023, -0.5090,  0.4993],
        [-0.1346, -0.5983, -0.1984],
        [ 0.6196,  0.0390, -0.1181],
        [ 0.9545,  0.4540,  1.1219]], grad_fn=<AddmmBackward>)


Backward hooks are functions that take three arguments: the layer it's applied to, the gradients of the loss with respect to the input, and the gradients with respect to the output.

In [ ]:
def example_backward_hook(m,gi,go): print(m,gi,go)
hook = tst_model.register_backward_hook(example_backward_hook)

x = torch.randn(4,5)
y = tst_model(x)
loss = y.pow(2).mean()
loss.backward()
hook.remove()

Linear(in_features=5, out_features=3, bias=True) (tensor([-0.0286, -0.3043,  0.0128]), None, tensor([[-0.0212,  0.0054,  0.1909],
        [-0.0305,  0.0262, -0.1591],
        [ 0.2990,  0.2336,  0.2875],
        [ 0.1074,  0.2575, -0.0094],
        [-0.1396,  0.1546, -0.2408]])) (tensor([[-0.1138, -0.1351, -0.0913],
        [ 0.1100,  0.0024,  0.0177],
        [-0.0442, -0.0869, -0.0727],
        [ 0.0192, -0.0846,  0.1590]]),)


Hooks can change the input/output of a layer, or the gradients, print values or shapes. If you want to store something related to theses inputs/outputs, it's best to have your hook associated to a class so that it can put it in the state of an instance of that class.

## Hook -

In [ ]:
#export
@docs
class Hook():
    "Create a hook on `m` with `hook_func`."
    def __init__(self, m, hook_func, is_forward=True, detach=True, cpu=False, gather=False):
        store_attr(self,'hook_func,detach,cpu,gather')
        f = m.register_forward_hook if is_forward else m.register_backward_hook
        self.hook = f(self.hook_fn)
        self.stored,self.removed = None,False

    def hook_fn(self, module, input, output):
        "Applies `hook_func` to `module`, `input`, `output`."
        if self.detach:
            input,output = to_detach(input, cpu=self.cpu, gather=self.gather),to_detach(output, cpu=self.cpu, gather=self.gather)
        self.stored = self.hook_func(module, input, output)

    def remove(self):
        "Remove the hook from the model."
        if not self.removed:
            self.hook.remove()
            self.removed=True

    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()

    _docs = dict(__enter__="Register the hook",
                 __exit__="Remove the hook")

This will be called during the forward pass if `is_forward=True`, the backward pass otherwise, and will optionally `detach`, `gather` and put on the `cpu` the (gradient of the) input/output of the model before passing them to `hook_func`. The result of `hook_func` will be stored in the `stored` attribute of the `Hook`.

In [ ]:
tst_model = nn.Linear(5,3)
hook = Hook(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hook.stored, y)

In [ ]:
show_doc(Hook.hook_fn)

<h4 id="Hook.hook_fn" class="doc_header"><code>Hook.hook_fn</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.hook_fn</code>(**`module`**, **`input`**, **`output`**)

Applies `hook_func` to [`module`](/layers#module), `input`, `output`.

In [ ]:
show_doc(Hook.remove)

<h4 id="Hook.remove" class="doc_header"><code>Hook.remove</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.remove</code>()

Remove the hook from the model.

> Note: It's important to properly remove your hooks for your model when you're done to avoid them being called again next time your model is applied to some inputs, and to free the memory that go with their state.

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
hook = Hook(tst_model, example_forward_hook)
test_stdout(lambda: tst_model(x), f"{tst_model} ({x},) {y.detach()}")
hook.remove()
test_stdout(lambda: tst_model(x), "")

### Context Manager

Since it's very important to remove your `Hook` even if your code is interrupted by some bug, `Hook` can be used as context managers.

In [ ]:
show_doc(Hook.__enter__)

<h4 id="Hook.__enter__" class="doc_header"><code>Hook.__enter__</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.__enter__</code>(**\*`args`**)

Register the hook

In [ ]:
show_doc(Hook.__exit__)

<h4 id="Hook.__exit__" class="doc_header"><code>Hook.__exit__</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>Hook.__exit__</code>(**\*`args`**)

Remove the hook

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
y = tst_model(x)
with Hook(tst_model, example_forward_hook) as h:
    test_stdout(lambda: tst_model(x), f"{tst_model} ({x},) {y.detach()}")
test_stdout(lambda: tst_model(x), "")

In [ ]:
#export
def _hook_inner(m,i,o): return o if isinstance(o,Tensor) or is_listy(o) else list(o)

def hook_output(module, detach=True, cpu=False, grad=False):
    "Return a `Hook` that stores activations of `module` in `self.stored`"
    return Hook(module, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `module`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
tst_model = nn.Linear(5,10)
x = torch.randn(4,5)
with hook_output(tst_model) as h:
    y = tst_model(x)
    test_eq(y, h.stored)
    assert not h.stored.requires_grad
    
with hook_output(tst_model, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    test_close(2*y / y.numel(), h.stored[0])

In [ ]:
#cuda
with hook_output(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    test_eq(h.stored.device, torch.device('cpu'))

## Hooks -

In [ ]:
#export
@docs
class Hooks():
    "Create several hooks on the modules in `ms` with `hook_func`."
    def __init__(self, ms, hook_func, is_forward=True, detach=True, cpu=False):
        self.hooks = [Hook(m, hook_func, is_forward, detach, cpu) for m in ms]

    def __getitem__(self,i): return self.hooks[i]
    def __len__(self):       return len(self.hooks)
    def __iter__(self):      return iter(self.hooks)
    @property
    def stored(self):        return L(o.stored for o in self)

    def remove(self):
        "Remove the hooks from the model."
        for h in self.hooks: h.remove()

    def __enter__(self, *args): return self
    def __exit__ (self, *args): self.remove()

    _docs = dict(stored = "The states saved in each hook.",
                 __enter__="Register the hooks",
                 __exit__="Remove the hooks")

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
hooks = Hooks(tst_model, lambda m,i,o: o)
y = tst_model(x)
test_eq(hooks.stored[0], layers[0](x))
test_eq(hooks.stored[1], F.relu(layers[0](x)))
test_eq(hooks.stored[2], y)
hooks.remove()

In [ ]:
show_doc(Hooks.stored, name='Hooks.stored')

<h4 id="Hooks.stored" class="doc_header"><code>Hooks.stored</code><a href="" class="source_link" style="float:right">[source]</a></h4>

The states saved in each hook.

In [ ]:
show_doc(Hooks.remove)

<h4 id="Hooks.remove" class="doc_header"><code>Hooks.remove</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.remove</code>()

Remove the hooks from the model.

### Context Manager

Like `Hook` , you can use `Hooks` as context managers.

In [ ]:
show_doc(Hooks.__enter__)

<h4 id="Hooks.__enter__" class="doc_header"><code>Hooks.__enter__</code><a href="__main__.py#L18" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.__enter__</code>(**\*`args`**)

Register the hooks

In [ ]:
show_doc(Hooks.__exit__)

<h4 id="Hooks.__exit__" class="doc_header"><code>Hooks.__exit__</code><a href="__main__.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>Hooks.__exit__</code>(**\*`args`**)

Remove the hooks

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
with Hooks(layers, lambda m,i,o: o) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)

In [ ]:
#export
def hook_outputs(modules, detach=True, cpu=False, grad=False):
    "Return `Hooks` that store activations of all `modules` in `self.stored`"
    return Hooks(modules, _hook_inner, detach=detach, cpu=cpu, is_forward=not grad)

The activations stored are the gradients if `grad=True`, otherwise the output of `modules`. If `detach=True` they are detached from their history, and if `cpu=True`, they're put on the CPU.

In [ ]:
layers = [nn.Linear(5,10), nn.ReLU(), nn.Linear(10,3)]
tst_model = nn.Sequential(*layers)
x = torch.randn(4,5)
with hook_outputs(layers) as h:
    y = tst_model(x)
    test_eq(h.stored[0], layers[0](x))
    test_eq(h.stored[1], F.relu(layers[0](x)))
    test_eq(h.stored[2], y)
    for s in h.stored: assert not s.requires_grad
    
with hook_outputs(layers, grad=True) as h:
    y = tst_model(x)
    loss = y.pow(2).mean()
    loss.backward()
    g = 2*y / y.numel()
    test_close(g, h.stored[2][0])
    g = g @ layers[2].weight.data
    test_close(g, h.stored[1][0])
    g = g * (layers[0](x) > 0).float()
    test_close(g, h.stored[0][0])

In [ ]:
#cuda
with hook_outputs(tst_model, cpu=True) as h:
    y = tst_model.cuda()(x.cuda())
    for s in h.stored: test_eq(s.device, torch.device('cpu'))

In [ ]:
#export
def dummy_eval(m, size=(64,64)):
    "Evaluate `m` on a dummy input of a certain `size`"
    ch_in = in_channels(m)
    x = one_param(m).new(1, ch_in, *size).requires_grad_(False).uniform_(-1.,1.)
    with torch.no_grad(): return m.eval()(x)

In [ ]:
#export
def model_sizes(m, size=(64,64)):
    "Pass a dummy input through the model `m` to get the various sizes of activations."
    with hook_outputs(m) as hooks:
        _ = dummy_eval(m, size=size)
        return [o.stored.shape for o in hooks]

In [ ]:
m = nn.Sequential(ConvLayer(3, 16), ConvLayer(16, 32, stride=2), ConvLayer(32, 32))
test_eq(model_sizes(m), [[1, 16, 64, 64], [1, 32, 32, 32], [1, 32, 32, 32]])

In [ ]:
#export
def num_features_model(m):
    "Return the number of output features for `m`."
    sz,ch_in = 32,in_channels(m)
    while True:
        #Trying for a few sizes in case the model requires a big input size.
        try:
            return model_sizes(m, (sz,sz))[-1][1]
        except Exception as e:
            sz *= 2
            if sz > 2048: raise e

In [ ]:
m = nn.Sequential(nn.Conv2d(5,4,3), nn.Conv2d(4,3,3))
test_eq(num_features_model(m), 3)
m = nn.Sequential(ConvLayer(3, 16), ConvLayer(16, 32, stride=2), ConvLayer(32, 32))
test_eq(num_features_model(m), 32)

## HookCallback -

To make hooks easy to use, we wrapped a version in a Callback where you just have to implement a `hook` function (plus any element you might need).

In [ ]:
#export
def has_params(m):
    "Check if `m` has at least one parameter"
    return len(list(m.parameters())) > 0

In [ ]:
assert has_params(nn.Linear(3,4))
assert has_params(nn.LSTM(4,5,2))
assert not has_params(nn.ReLU())

In [ ]:
#export
@funcs_kwargs
class HookCallback(Callback):
    "`Callback` that can be used to register hooks on `modules`"
    _methods = ["hook"]
    hook = noops
    def __init__(self, modules=None, every=None, remove_end=True, is_forward=True, detach=True, cpu=True, **kwargs):
        store_attr(self, 'modules,every,remove_end,is_forward,detach,cpu')
        assert not kwargs

    def begin_fit(self):
        "Register the `Hooks` on `self.modules`."
        if self.modules is None: self.modules = [m for m in flatten_model(self.model) if has_params(m)]
        if self.every is None: self._register()

    def begin_batch(self):
        if self.every is None: return
        if self.training and self.train_iter%self.every==0: self._register()

    def after_batch(self):
        if self.every is None: return
        if self.training and self.train_iter%self.every==0: self._remove()

    def after_fit(self):
        "Remove the `Hooks`."
        if self.remove_end: self._remove()

    def _register(self): self.hooks = Hooks(self.modules, self.hook, self.is_forward, self.detach, self.cpu)
    def _remove(self):
        if getattr(self, 'hooks', None): self.hooks.remove()

    def __del__(self): self._remove()

You can either subclass and implement a `hook` function (along with any event you want) or pass that a `hook` function when initializing. Such a function needs to take three argument: a layer, input and output (for a backward hook, input means gradient with respect to the inputs, output, gradient with respect to the output) and can either modify them or update the state according to them.

If not provided, `modules` will default to the layers of `self.model` that have a `weight` attribute. Depending on `do_remove`, the hooks will be properly removed at the end of training (or in case of error). `is_forward` , `detach` and `cpu` are passed to `Hooks`.

The function called at each forward (or backward) pass is `self.hook` and must be implemented when subclassing this callback.

In [ ]:
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def after_batch(self): test_eq(self.hooks.stored[0], self.pred)
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

(#4) [0,21.6732234954834,17.283117294311523,'00:00']


In [ ]:
class TstCallback(HookCallback):
    def __init__(self, modules=None, remove_end=True, detach=True, cpu=False):
        super().__init__(modules, None, remove_end, False, detach, cpu)
    def hook(self, m, i, o): return o
    def after_batch(self):
        if self.training:
            test_eq(self.hooks.stored[0][0], 2*(self.pred-self.y)/self.pred.shape[0])
        
learn = synth_learner(n_trn=5, cbs = TstCallback())
learn.fit(1)

(#4) [0,17.65213394165039,16.966201782226562,'00:00']


In [ ]:
show_doc(HookCallback.begin_fit)

<h4 id="HookCallback.begin_fit" class="doc_header"><code>HookCallback.begin_fit</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>HookCallback.begin_fit</code>()

Register the [`Hooks`](/callback.hook#Hooks) on `self.modules`.

In [ ]:
show_doc(HookCallback.after_fit)

<h4 id="HookCallback.after_fit" class="doc_header"><code>HookCallback.after_fit</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>HookCallback.after_fit</code>()

Remove the [`Hooks`](/callback.hook#Hooks).

## Model summary

In [ ]:
#export
def total_params(m):
    "Give the number of parameters of a module and if it's trainable or not"
    params = sum([p.numel() for p in m.parameters()])
    trains = [p.requires_grad for p in m.parameters()]
    return params, (False if len(trains)==0 else trains[0])

In [ ]:
test_eq(total_params(nn.Linear(10,32)), (32*10+32,True))
test_eq(total_params(nn.Linear(10,32, bias=False)), (32*10,True))
test_eq(total_params(nn.BatchNorm2d(20)), (20*2, True))
test_eq(total_params(nn.BatchNorm2d(20, affine=False)), (0,False))
test_eq(total_params(nn.Conv2d(16, 32, 3)), (16*32*3*3 + 32, True))
test_eq(total_params(nn.Conv2d(16, 32, 3, bias=False)), (16*32*3*3, True))
#First ih layer 20--10, all else 10--10. *4 for the four gates
test_eq(total_params(nn.LSTM(20, 10, 2)), (4 * (20*10 + 10) + 3 * 4 * (10*10 + 10), True))

In [ ]:
#export
def layer_info(learn):
    def _track(m, i, o):
        return (m.__class__.__name__,)+total_params(m)+(apply(lambda x:x.shape, o),)
    layers = [m for m in flatten_model(learn.model)]
    xb = learn.dls.train.one_batch()[:learn.dls.train.n_inp]
    with Hooks(layers, _track) as h:
        _ = learn.model.eval()(*apply(lambda o:o[:1], xb))
        return xb,h.stored

In [ ]:
m = nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
learn = synth_learner()
learn.model=m

In [ ]:
test_eq(layer_info(learn)[1], [
    ('Linear', 100, True, [1, 50]),
    ('ReLU', 0, False, [1, 50]),
    ('BatchNorm1d', 100, True, [1, 50]),
    ('Linear', 51, True, [1, 1])
])

In [ ]:
# Test for n_inp
def _ninp_synth_dls(n=160, bs=16, n_inp=1, cuda=False):
    def get_data(n, n_inp):
        xs = [torch.randn(n, 1) for _ in range(n_inp)] 
        y = torch.cat(xs, dim=-1).sum(-1)
        return torch.utils.data.TensorDataset(*xs, y)
    
    n_train = int(n*0.8)
    train_ds = get_data(n_train, n_inp)
    valid_ds = get_data(n-n_train, n_inp)
    device = default_device() if cuda else None
    train_dl = TfmdDL(train_ds, bs=bs, shuffle=True, num_workers=0)
    valid_dl = TfmdDL(valid_ds, bs=bs, num_workers=0)
    return DataLoaders(train_dl, valid_dl, device=device)

class _NInpModel(Module):
    def __init__(self, n_inp=1):
        super().__init__()
        self.n_inp = n_inp
        self.seq = nn.Sequential(nn.Linear(1*n_inp,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
    def forward(self, *inps):
        outputs = torch.cat(inps, dim=-1)
        return self.seq(outputs)

n_inp = 3
dls = _ninp_synth_dls(n_inp=n_inp)
m = _NInpModel(n_inp)
learn = Learner(dls, m, lr=1e-3, loss_func=MSELossFlat(), opt_func=partial(SGD, mom=0.9))

test_eq(layer_info(learn)[1], [
    ('Linear', 200, True, [1, 50]),
    ('ReLU', 0, False, [1, 50]),
    ('BatchNorm1d', 100, True, [1, 50]),
    ('Linear', 51, True, [1, 1])
])

In [ ]:
#export
def _print_shapes(o, bs):
    if isinstance(o, torch.Size): return ' x '.join([str(bs)] + [str(t) for t in o[1:]])
    else: return str([_print_shapes(x, bs) for x in o])

In [ ]:
#hide
#Individual parameters wrapped in ParameterModule aren't called through the hooks in `layer_info`, thus are not counted inside the summary
#TODO: find a way to have them counted in param number somehow

In [ ]:
#export
@patch
def summary(self:Learner):
    "Print a summary of the model, optimizer and loss function."
    xb,infos = layer_info(self)
    n,bs = 64,find_bs(xb)
    inp_sz = _print_shapes(apply(lambda x:x.shape, xb), bs)
    res = f"{self.model.__class__.__name__} (Input shape: {inp_sz})\n"
    res += "=" * n + "\n"
    res += f"{'Layer (type)':<20} {'Output Shape':<20} {'Param #':<10} {'Trainable':<10}\n"
    res += "=" * n + "\n"
    ps,trn_ps = 0,0
    infos = [o for o in infos if o is not None] #see comment in previous cell
    for typ,np,trn,sz in infos:
        if sz is None: continue
        ps += np
        if trn: trn_ps += np
        res += f"{typ:<20} {_print_shapes(sz, bs)[:19]:<20} {np:<10,} {str(trn):<10}\n"
        res += "_" * n + "\n"
    res += f"\nTotal params: {ps:,}\n"
    res += f"Total trainable params: {trn_ps:,}\n"
    res += f"Total non-trainable params: {ps - trn_ps:,}\n\n"
    res += f"Optimizer used: {self.opt_func}\nLoss function: {self.loss_func}\n\n"
    if self.opt is not None:
        res += f"Model " + ("unfrozen\n\n" if self.opt.frozen_idx==0 else f"frozen up to parameter group number {self.opt.frozen_idx}\n\n")
    res += "Callbacks:\n" + '\n'.join(f"  - {cb}" for cb in sort_by_run(self.cbs))
    return PrettyString(res)

In [ ]:
m = nn.Sequential(nn.Linear(1,50), nn.ReLU(), nn.BatchNorm1d(50), nn.Linear(50, 1))
for p in m[0].parameters(): p.requires_grad_(False)
learn = synth_learner()
learn.create_opt()
learn.model=m
learn.summary()

Sequential (Input shape: ['16 x 1'])
Layer (type)         Output Shape         Param #    Trainable 
Linear               16 x 50              100        False     
________________________________________________________________
ReLU                 16 x 50              0          False     
________________________________________________________________
BatchNorm1d          16 x 50              100        True      
________________________________________________________________
Linear               16 x 1               51         True      
________________________________________________________________

Total params: 251
Total trainable params: 151
Total non-trainable params: 100

Optimizer used: functools.partial(<function SGD at 0x7f62408eb290>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Model unfrozen

Callbacks:
  - TrainEvalCallback
  - Recorder

In [ ]:
# Test for multiple output
class _NOutModel(nn.Module):
    def forward(self, x1):
        seq_len, bs, hid_size = 50, 16, 256
        num_layer = 1
        return torch.randn((seq_len, bs, hid_size)), torch.randn((num_layer, bs, hid_size))
m = _NOutModel()
learn = synth_learner()
learn.model = m
learn.summary() # Output Shape should be (50, 16, 256), (1, 16, 256)

_NOutModel (Input shape: ['16 x 1'])
Layer (type)         Output Shape         Param #    Trainable 
_NOutModel           ['16 x 16 x 256', '  0          False     
________________________________________________________________

Total params: 0
Total trainable params: 0
Total non-trainable params: 0

Optimizer used: functools.partial(<function SGD at 0x7f62408eb290>, mom=0.9)
Loss function: FlattenedLoss of MSELoss()

Callbacks:
  - TrainEvalCallback
  - Recorder

## Activation graphs

This is an example of a `HookCallback`, that stores the mean, stds and histograms of activations that go through the network.

In [ ]:
#exports
@delegates()
class ActivationStats(HookCallback):
    "Callback that record the mean and std of activations."
    run_before=TrainEvalCallback
    def __init__(self, with_hist=False, **kwargs):
        super().__init__(**kwargs)
        self.with_hist = with_hist

    def begin_fit(self):
        "Initialize stats."
        super().begin_fit()
        self.stats = L()

    def hook(self, m, i, o):
        o = o.float()
        res = {'mean': o.mean().item(), 'std': o.std().item(),
               'near_zero': (o<=0.05).long().sum().item()/o.numel()}
        if self.with_hist: res['hist'] = o.histc(40,0,10)
        return res

    def after_batch(self):
        "Take the stored results and puts it in `self.stats`"
        if self.training and (self.every is None or self.train_iter%self.every == 0):
            self.stats.append(self.hooks.stored)
        super().after_batch()
        
    def layer_stats(self, idx):
        lstats = self.stats.itemgot(idx)
        return L(lstats.itemgot(o) for o in ('mean','std','near_zero'))
    
    def hist(self, idx):
        res = self.stats.itemgot(idx).itemgot('hist')
        return torch.stack(tuple(res)).t().float().log1p()

    def plot_hist(self, idx, figsize=(10,5), ax=None):
        res = self.hist(idx)
        if ax is None: ax = subplots(figsize=figsize)[1][0]
        ax.imshow(res, origin='lower')
        ax.axis('off')

    def plot_layer_stats(self, idx):
        _,axs = subplots(1, 3, figsize=(12,3))
        for o,ax,title in zip(self.layer_stats(idx),axs,('mean','std','% near zero')):
            ax.plot(o)
            ax.set_title(title)

In [ ]:
learn = synth_learner(n_trn=5, cbs = ActivationStats(every=4))
learn.fit(1)

(#4) [0,19.558387756347656,10.741193771362305,'00:00']


In [ ]:
learn.activation_stats.stats

(#2) [(#1) [{'mean': 0.11941853910684586, 'std': 1.0692791938781738, 'near_zero': 0.5}],(#1) [{'mean': 0.34276124835014343, 'std': 1.1721261739730835, 'near_zero': 0.4375}]]

In [ ]:
import math

def test_every(n_tr, every):
    "create a learner, fit, then check number of stats collected"
    learn = synth_learner(n_trn=n_tr, cbs=ActivationStats(every=every))
    learn.fit(1)
    expected_stats_len = math.ceil(n_tr / every)
    test_eq(expected_stats_len, len(learn.activation_stats.stats))
    
for n_tr in [11, 12, 13]:
    test_every(n_tr, 4)
    test_every(n_tr, 1)

(#4) [0,11.054356575012207,11.05624771118164,'00:00']
(#4) [0,10.04439926147461,6.602774143218994,'00:00']
(#4) [0,8.677932739257812,6.921849250793457,'00:00']
(#4) [0,9.601393699645996,9.777692794799805,'00:00']
(#4) [0,31.271684646606445,33.13401794433594,'00:00']
(#4) [0,17.567604064941406,10.449918746948242,'00:00']


The first line contains the means of the outputs of the model for each batch in the training set, the second line their standard deviations.

In [ ]:
#hide
class TstCallback(HookCallback):
    def hook(self, m, i, o): return o
    def begin_fit(self):
        super().begin_fit()
        self.means,self.stds = [],[]
    
    def after_batch(self):
        if self.training:
            self.means.append(self.hooks.stored[0].mean().item())
            self.stds.append (self.hooks.stored[0].std() .item())

learn = synth_learner(n_trn=5, cbs = [TstCallback(), ActivationStats()])
learn.fit(1)
test_eq(learn.activation_stats.stats.itemgot(0).itemgot("mean"), learn.tst.means)
test_eq(learn.activation_stats.stats.itemgot(0).itemgot("std"),  learn.tst.stds)

(#4) [0,7.901733875274658,13.594461441040039,'00:00']


## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.learner.ipynb.
Converted 22_tutorial.imagenette.ipynb.
Converted 23_tutoria